In [35]:
import random
import gurobipy as grb
import math
import numpy as np

n = 4
Distance = 50000000
# Distance = 200

# def distance(points, i, j):
#   dx = points[i][0] - points[j][0]
#   dy = points[i][1] - points[j][1]
#   return math.sqrt(dx*dx + dy*dy)

# random.seed(1)
# points = []
# for i in range(n):
#   points.append((random.randint(0,100),random.randint(0,100)))

points = [(i,j) for i in range(6) for j in range(6)]
distance = np.random.randint(0, 100, size=(6,6))

opt_model = grb.Model(name="MILP Model")

# <= Variables
x_vars = {}
for i in range(n):
   for j in range(n):
     x_vars[i,j] = opt_model.addVar(vtype=grb.GRB.BINARY,
                          name='e'+str(i)+'_'+str(j))
u={}
for i in range(1,n):
    u[i]=opt_model.addVar(vtype=grb.GRB.INTEGER,
                          name='e'+str(i))

# <= Constraint (Mandatory Edges and excluding vertexes) Eq(1)


opt_model.addConstr(grb.quicksum([x_vars[1,j] for j in range(1,n)])  == 1)
opt_model.addConstr(grb.quicksum([x_vars[i,n-1] for i in range(n-1)])  == 1)
# opt_model.addConstr((grb.quicksum(x_vars[i,i] for i in range(n-1)))  == 0)
# <= Constraint (Distance) Eq(3)

# for i in range(n-1):
#   opt_model.addConstr(grb.quicksum([x_vars[i,j]*distance(points, i, j) for j in range(1,n)]) <= Distance)

# opt_model.addConstr(grb.quicksum([x_vars[i,j]*distance(points, i,j) for j in range(1,n) for i in range(n-1)]) <= Distance)
opt_model.addConstr(grb.quicksum([x_vars[i,j]*distance[i,j] for j in range(1,n) for i in range(n-1)]) <= Distance)

# <= Constraint (Equality & Single edge in and out) Eq(2)

for k in range(1, n-1):
#   opt_model.addConstr(grb.quicksum(x_vars[i,k] for i in range(n-1))
#                       == grb.quicksum(x_vars[k,j] for j in range(1, n)) <=1)

    one = grb.quicksum([x_vars[i,k] for i in range(n-1)])
    two = grb.quicksum([x_vars[k,j] for j in range(1, n)])
    opt_model.addConstr(one == two)
    opt_model.addConstr(one <= 1)

# <= Constraint (Subtour elimination) Eq(4) Eq(5)

for i in range(1,n):
    opt_model.addConstr(2 <= u[i])
    opt_model.addConstr(u[i] <= n)
#   opt_model.addConstr(2 <= u[i] <= n)

for i in range(1,n):
    for j in range(1,n):
        opt_model.addConstr((u[i] - u[j] +1 <= (n-1)*(1-x_vars[j,i])))

# <= objective (maximize) Eq(1)

objective = grb.quicksum([x_vars[i,j]
                         for i in range(1, n-1)
                         for j in range(1, n)])

opt_model.ModelSense = grb.GRB.MAXIMIZE
opt_model.setObjective(objective)
opt_model.optimize()


# opt_model.update()
# opt_model.computeIIS()
# opt_model.write("model.ilp")
solution = opt_model.getAttr('x', x_vars )

print (solution)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 22 rows, 19 columns and 56 nonzeros
Model fingerprint: 0x21bb240c
Variable types: 0 continuous, 19 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+07]
Found heuristic solution: objective 1.0000000
Presolve removed 22 rows and 19 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
{(0, 0): 0.0, (0, 1): 1.0, (0, 2): 0.0, (0, 3): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 1.0, (1, 3): 0.0, (2, 0): 0.0, (2, 1): -0.0, (2, 2): 0.0, (2, 3): 1.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0}


In [36]:
select = grb.tuplelist((i,j) for i,j in x_vars.keys() if x_vars[i,j].X > 0.5)
print (select)

<gurobi.tuplelist (3 tuples, 2 values each):
 ( 0 , 1 )
 ( 1 , 2 )
 ( 2 , 3 )
>


In [38]:
total = 0
for z in range(len(select)):
#     total += distance(points, select[z][0], select[z][1])
    total += distance[select[z][0], select[z][1]]
    
print (total)

177


In [39]:
points

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5)]